In [1]:
#from google.colab import files, drive
#drive.mount('/content/drive')

In [2]:
#!pip install -r ./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/requirements.txt

In [3]:

from __future__ import absolute_import

import sys
import os

sys.path.append('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/')


In [4]:
#!wandb login

In [5]:
from __future__ import absolute_import

import sys
import os

import shutil

try:
    from dotenv import find_dotenv, load_dotenv
except:
    pass

import argparse

try:
    sys.path.append(os.path.join(os.path.dirname(__file__), './drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/'))
except:
    sys.path.append(os.path.join(os.getcwd(), './drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/'))
    
try:
    sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
except:
    sys.path.append(os.path.join(os.getcwd(), '../'))
    
import pandas as pd
import numpy as np

import pickle
from collections import Counter
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

try:
    import wandb
    load_dotenv(find_dotenv())
    wandb.login(key=os.environ['WANDB_API_KEY'])
    from wandb.keras import WandbCallback
    _has_wandb = True
except:
    _has_wandb = False

import tokenizers
from transformers import TFAutoModel, AutoTokenizer, AutoConfig, BertTokenizer

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from src import data, models
from src.models.models import *

pd.options.display.max_colwidth = -1

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING Calling wandb.login() without arguments from jupyter should prompt you for an api key.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/victor/.netrc
I0122 13:29:06.285727 4630683136 file_utils.py:41] PyTorch version 1.4.0 available.
I0122 13:29:06.288890 4630683136 file_utils.py:57] TensorFlow version 2.2.0-rc3 available.
/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [6]:
print (_has_wandb)

True


In [7]:
parser = argparse.ArgumentParser(prog='Trainer',conflict_handler='resolve')

#parser.add_argument('--train_data', type=str, default='./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/raw/Hate-speech-dataset/hate_speech.tsv', required=False,
#                    help='train data')
#parser.add_argument('--train_data', type=str, default='../data/raw/Hate-speech-dataset/hate_speech.tsv', required=False,
#                    help='train data')
#parser.add_argument('--data_dir', type=str, default='./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/', required=False,
#                    help='train data')

parser.add_argument('--data_dir', type=str, default='../data/', required=False,
                    help='train data')

parser.add_argument('--val_data', type=str, default=None, required=False,
                    help='validation data')
parser.add_argument('--test_data', type=str, default=None, required=False,
                    help='test data')

parser.add_argument('--transformer_model_pretrained_path', type=str, default='roberta-base', required=False,
                    help='transformer model pretrained path or huggingface model name')
parser.add_argument('--transformer_config_path', type=str, default='roberta-base', required=False,
                    help='transformer config file path or huggingface model name')
parser.add_argument('--transformer_tokenizer_path', type=str, default='roberta-base', required=False,
                    help='transformer tokenizer file path or huggingface model name')

parser.add_argument('--max_text_len', type=int, default=50, required=False,
                    help='maximum length of text')
parser.add_argument('--max_char_len', type=int, default=200, required=False,
                    help='maximum length of text')
parser.add_argument('--max_word_char_len', type=int, default=20, required=False,
                    help='maximum length of text')

parser.add_argument('--emb_dim', type=int, default=128, required=False,
                    help='maximum length of text')
parser.add_argument('--n_layers', type=int, default=2, required=False,
                    help='maximum length of text')
parser.add_argument('--n_units', type=int, default=128, required=False,
                    help='maximum length of text')

parser.add_argument('--epochs', type=int, default=500, required=False,
                    help='number of epochs')
parser.add_argument('--lr', type=float, default=.001, required=False,
                    help='learning rate')
parser.add_argument('--early_stopping_rounds', type=int, default=50, required=False,
                    help='number of epochs for early stopping')
parser.add_argument('--lr_schedule_round', type=int, default=30, required=False,
                    help='number of epochs for learning rate scheduling')

parser.add_argument('--train_batch_size', type=int, default=32, required=False,
                    help='train batch size')
parser.add_argument('--eval_batch_size', type=int, default=16, required=False,
                    help='eval batch size')

#parser.add_argument('--model_save_path', type=str, default='./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/models/model_hindi_sentiment/', required=False,
#                    help='seed')

parser.add_argument('--model_save_path', type=str, default='../models/hate_detection/', required=False,
                    help='seed')

parser.add_argument('--wandb_logging', type=bool, default=True, required=False,
                    help='wandb logging needed')

parser.add_argument('--seed', type=int, default=42, required=False,
                    help='seed')


args, _ = parser.parse_known_args()

In [8]:
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

In [9]:
#df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/raw/Hate-speech-dataset/hate_speech.tsv', \
#                 sep='\t',header=None,usecols=[0,1])
df = pd.read_csv(os.path.join(args.data_dir,'raw','Hate-speech-dataset','hate_speech.tsv'), \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'hate'

hate_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

hate_val_df = df.iloc[val_index]
hate_test_df = df.iloc[test_index]

#df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/StanceDetection_CodeMixed/data.txt', \
#                 sep='\t',header=None,usecols=[0,1])
df = pd.read_csv(os.path.join(args.data_dir,'processed','StanceDetection_CodeMixed', 'data.txt'), \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'stance'
stance_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

stance_val_df = df.iloc[val_index]
stance_test_df = df.iloc[test_index]

#df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/SarcasmDetection_CodeMixed/data.txt', \
#                 sep='\t',header=None,usecols=[0,1])
df = pd.read_csv(os.path.join(args.data_dir,'processed','SarcasmDetection_CodeMixed','data.txt'), \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'sarcasm'

sarcasm_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

sarcasm_val_df = df.iloc[val_index]
sarcasm_test_df = df.iloc[test_index]

#df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/humor-detection-corpus/data.txt', \
#                 sep='\t',header=None,usecols=[0,1])
df = pd.read_csv(os.path.join(args.data_dir,'processed','humor-detection-corpus','data.txt'), \
                 sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'humor'

humor_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

humor_val_df = df.iloc[val_index]
humor_test_df = df.iloc[test_index]

#df = pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/Aggression_dataset/train.txt', \
#                 sep='\t',header=None,usecols=[0,1])
#df = pd.concat([df,pd.read_csv('./drive/MyDrive/Hate_detection/Hinglish_Hate_Detection/data/processed/Aggression_dataset/val.txt', \
#                 sep='\t',header=None,usecols=[0,1])],axis=0)
df = pd.read_csv(os.path.join(args.data_dir,'processed','Aggression_dataset','train.txt'), \
                 sep='\t',header=None,usecols=[0,1])
df = pd.concat([df,pd.read_csv(os.path.join(args.data_dir,'processed','Aggression_dataset','val.txt'), \
                 sep='\t',header=None,usecols=[0,1])],axis=0)

df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

df['type'] = 'aggression'

aggression_train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

aggression_val_df = df.iloc[val_index]
aggression_test_df = df.iloc[test_index]


I0122 13:29:13.430094 4630683136 utils.py:141] NumExpr defaulting to 4 threads.


In [10]:
train_df = pd.concat([hate_train_df,stance_train_df,sarcasm_train_df,humor_train_df,aggression_train_df],axis=0)
val_df = pd.concat([hate_val_df,stance_val_df,sarcasm_val_df,humor_val_df,aggression_val_df],axis=0)
test_df = pd.concat([hate_test_df,stance_test_df,sarcasm_test_df,humor_test_df,aggression_test_df],axis=0)

print (train_df.shape, val_df.shape, test_df.shape)

(37059, 3) (4632, 3) (4634, 3)


In [11]:
train_df.tail(5)

,text,category,type
5995,Great stand by father but the guy needs a burial???,CAG,aggression
5996,"PAYTM is a unscrupulous vendor see my experience : I Paid through Paytm ( order no. 2532865244) payment of Rs.590/- towards my CESC Electricity Bill for CESC Consumer ID-60000208248 on January 6 morning, but after 5 days , it is still showing PROCESSING, I have raised numerous tickets, but no one is there to reply me. Is it a reliable service ? or birth of mistrust ? I wonder how our Central Government is promoting such scrap?",CAG,aggression
5998,Just saw this news\n* read 1dt comment. \n*smiled and thought it must be ndtv or hindustantimes. \n* scrolled up and checked.. Ndtv. \n* laughed and said saeed.. Ye madharchod nhi sudrege. \n*moved on.,CAG,aggression
5999,U guys can crack jokes but can't take the cridibility of Indian armed force.... already war with Pak is one sided ie 4 -0 ....,CAG,aggression
6000,"These media fellows have time to dissect one's social media profile,but none of spineless jokers has the guts to question as to why and how a convict like Lalu Yadav is roaming outside and running a Government.",CAG,aggression


In [12]:
train_df.text = train_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
val_df.text = val_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
test_df.text = test_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))

train_df = train_df[train_df.text != '']
val_df = val_df[val_df.text != '']
test_df = test_df[test_df.text != '']

In [13]:
hate_train_df.text = hate_train_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
hate_val_df.text = hate_val_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
hate_test_df.text = hate_test_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))

hate_train_df = hate_train_df[hate_train_df.text != '']
hate_val_df = hate_val_df[hate_val_df.text != '']
hate_test_df = hate_test_df[hate_test_df.text != '']


In [14]:
train_df.text.apply(lambda x: len(x)).describe()

count    37048.000000
mean     122.742307  
std      175.689032  
min      1.000000    
25%      57.000000   
50%      89.000000   
75%      131.000000  
max      6867.000000 
Name: text, dtype: float64

In [15]:
train_df.text.apply(lambda x: len(x.split())).describe()

count    37048.000000
mean     22.966746   
std      32.452479   
min      1.000000    
25%      11.000000   
50%      17.000000   
75%      25.000000   
max      1124.000000 
Name: text, dtype: float64

In [16]:
model_save_dir = args.model_save_path

try:
    os.makedirs(model_save_dir)
except OSError:
    pass

In [17]:
hate_train_df.category.value_counts()

no     2316
yes    1342
on     2   
n      1   
Name: category, dtype: int64

In [18]:
#train_df = train_df[train_df.category.str.contains('yes|no')]
#val_df = val_df[val_df.category.str.contains('yes|no')]
#test_df = test_df[test_df.category.str.contains('yes|no')]

In [19]:
hate_train_df = hate_train_df[hate_train_df.category.str.contains('yes|no')]
hate_val_df = hate_val_df[hate_val_df.category.str.contains('yes|no')]
hate_test_df = hate_test_df[hate_test_df.category.str.contains('yes|no')]

In [20]:
model_save_dir

'../models/hate_detection/'

In [21]:
hate_train_df.category, label2idx = data.data_utils.convert_categorical_label_to_int(hate_train_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

hate_val_df.category, _ = data.data_utils.convert_categorical_label_to_int(hate_val_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

hate_test_df.category, _ = data.data_utils.convert_categorical_label_to_int(hate_test_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

In [22]:
print (label2idx)

{'no': 0, 'yes': 1}


In [23]:
idx2label = {i:w for (w,i) in label2idx.items()}

### Learn tokenizer

In [24]:
#data.custom_tokenizers.custom_wp_tokenizer(hate_train_df.text.values, args.model_save_path, args.model_save_path)
tokenizer = BertTokenizer.from_pretrained(args.model_save_path)

I0122 13:29:19.011621 4630683136 tokenization_utils.py:420] Model name '../models/hate_detection/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '../models/hate_detection/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0122 13:29:19.014900 4630683136 tokenization_utils.py:449] Didn't find file ../models/hate_detection/added_tokens.json. We won't load it.
I0122 13:29:19.015823 4630683136 tokenization_utils.py:449] Didn't

In [25]:
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, split=' ',oov_token=1)
char_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, split='',oov_token=1)

word_tokenizer.fit_on_texts(hate_train_df.text.values)
char_tokenizer.fit_on_texts(hate_train_df.text.values)

### Modeling

In [26]:
n_words = len(word_tokenizer.word_index)+1
n_chars = len(char_tokenizer.word_index)+1
n_subwords = tokenizer.vocab_size
tfidf_shape = None #train_tfidf.shape[1]
n_out = len(label2idx)

In [27]:
model = HAN(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords, n_out=n_out,max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)

model.load_weights(os.path.join(args.model_save_path,'HAN_ce_without_features.h5'))

In [28]:
def get_conditional_weights(text, model, class_idx):
    #all_texts = [" ".join(text.split()[:i]) for i in range(len(text.split())+1)][1:]
    all_texts = [text]*(len(text.split())+1)
    word_train_inputs = word_tokenizer.texts_to_sequences(all_texts)
    word_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_train_inputs, maxlen=args.max_text_len)

    subword_train_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                            for text in all_texts])

    char_train_inputs = char_tokenizer.texts_to_sequences(all_texts)
    char_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_train_inputs, maxlen=args.max_char_len)
    
    for i in range(len(text.split())):
        #subword_train_inputs_ = subword_train_inputs.copy()
        subword_train_inputs[i,i,:] = np.array([0]*subword_train_inputs.shape[-1])
        
    #print (subword_train_inputs[-1])
    probs = model.predict([word_train_inputs,char_train_inputs,subword_train_inputs,\
                                      char_train_inputs])
    
    probs = probs[-1] - probs
    #class_idx = probs.argmax(-1)[-1]
    
    return probs[:-1,class_idx]

In [29]:
word_causal_weights = []
words = []

causal_weights_df = pd.DataFrame()

for text in tqdm(pd.concat([hate_train_df, hate_val_df, hate_test_df], axis=0).text.values):
    text = " ".join(text.split()[:args.max_text_len])
    weights = get_conditional_weights(text,model,1)
    for i, word in enumerate(text.split()):
        words.append(word)
        word_causal_weights.append(weights[i])
        
causal_weights_df['word'] = words
causal_weights_df['Hate_yes'] = word_causal_weights

100%|██████████| 4573/4573 [1:47:51<00:00,  1.42s/it]  


In [30]:
#causal_weights_df = causal_weights_df.groupby(['word'])['Hate_yes'].mean().reset_index(drop=False)
causal_weights_df.sort_values(['Hate_yes'],ascending=[False]).head(10)

,word,Hate_yes
11803,he,1.0
78936,he,1.0
87416,i,1.0
11802,gyei.,1.0
9242,i,1.0
51787,karte,1.0
65378,rape,1.0
78934,nikal,1.0
78935,gyei.,1.0
53188,i,1.0


In [31]:
causal_weights_df.to_csv(os.path.join(args.model_save_path,'word_weights.csv'),index=False,sep='\t')

In [ ]:
train_df = train_df.sample(frac=1)

In [50]:
transformer_train_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(train_df, 'text', tokenizer, args.max_char_len)

word_train_inputs = word_tokenizer.texts_to_sequences(train_df.text.values)
word_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_train_inputs, maxlen=args.max_text_len)

subword_train_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(train_df.text.values)])

char_train_inputs = char_tokenizer.texts_to_sequences(train_df.text.values)
char_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_train_inputs, maxlen=args.max_char_len)

100%|██████████| 37867/37867 [00:20<00:00, 1823.07it/s]


In [51]:
probs = model.predict([word_train_inputs,char_train_inputs,subword_train_inputs,\
                                      transformer_train_inputs])
train_df['category_prob'] = probs.max(-1)
train_df['category'] = probs.argmax(-1)

train_df = train_df[train_df.category_prob > .95]
train_df = train_df.sort_values(['category_prob'],ascending=[False]).reset_index(drop=True)

In [58]:
train_df['category'].value_counts()

,text,category,type
2,"doctor sab sahi me ke phd (in hate politics) wale. bhai padhe likhe ho fir kyu ye sab baate karte ho. tum bas bowling khelo, aur maje lo. pic.twitter.com/fkqubqstw",0,hate
3,"poore desh me patel obc me aate hain sirf gujrat ko chor kar may be, ye manuwadiyon bramanwadi kabhi aapko aarackchan nahi denge ye to jis obc ko mila hai usse bhi nafrat karte hain ye khoon aur chamdi ka frak karne waale bharmhanwadi kisi ke sage nahi hain",0,hate
5,hate it jab test ata ho phr bh acha na ho -.-,1,hate
6,ek dil ek jaan akal ke imitihaan kal kp ke rape aur sawing between two legs after rape honge justified due to cinematic liberty aaj toh rajput ki ho gayi shaan urdu ki gulaam,0,hate
7,aise logo se sakht nafrat karta hu jo caste ko naam ke sath jod ke chaude hote h but real me vo piddu hote h,1,hate


In [59]:
transformer_train_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(train_df, 'text', tokenizer, args.max_char_len)

word_train_inputs = word_tokenizer.texts_to_sequences(train_df.text.values)
word_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_train_inputs, maxlen=args.max_text_len)

subword_train_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(train_df.text.values)])

char_train_inputs = char_tokenizer.texts_to_sequences(train_df.text.values)
char_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_train_inputs, maxlen=args.max_char_len)

train_outputs = data.data_utils.compute_output_arrays(train_df, 'category')

transformer_val_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(hate_val_df, 'text', tokenizer, args.max_char_len)

word_val_inputs = word_tokenizer.texts_to_sequences(hate_val_df.text.values)
word_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_val_inputs, maxlen=args.max_text_len)

subword_val_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(hate_val_df.text.values)])

char_val_inputs = char_tokenizer.texts_to_sequences(hate_val_df.text.values)
char_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_val_inputs, maxlen=args.max_char_len)

val_outputs = data.data_utils.compute_output_arrays(hate_val_df, 'category')

transformer_test_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(hate_test_df, 'text', tokenizer, args.max_char_len)

word_test_inputs = word_tokenizer.texts_to_sequences(hate_test_df.text.values)
word_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_test_inputs, maxlen=args.max_text_len)

subword_test_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(hate_test_df.text.values)])

char_test_inputs = char_tokenizer.texts_to_sequences(hate_test_df.text.values)
char_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_test_inputs, maxlen=args.max_char_len)

full_transformer_test_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(test_df, 'text', tokenizer, args.max_char_len)

full_word_test_inputs = word_tokenizer.texts_to_sequences(test_df.text.values)
full_word_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(full_word_test_inputs, maxlen=args.max_text_len)

full_subword_test_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(test_df.text.values)])

full_char_test_inputs = char_tokenizer.texts_to_sequences(test_df.text.values)
full_char_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(full_char_test_inputs, maxlen=args.max_char_len)

test_outputs = data.data_utils.compute_output_arrays(hate_test_df, 'category')

train_outputs = tf.keras.utils.to_categorical(train_outputs, \
                                                    num_classes=len(label2idx))
val_outputs = tf.keras.utils.to_categorical(val_outputs, \
                                                    num_classes=len(label2idx))
test_outputs = tf.keras.utils.to_categorical(test_outputs, \
                                                    num_classes=len(label2idx))

print (transformer_train_inputs.shape, subword_train_inputs.shape, word_train_inputs.shape, char_train_inputs.shape, \
       train_outputs.shape)
print (transformer_val_inputs.shape, subword_val_inputs.shape, word_val_inputs.shape, char_val_inputs.shape, \
       val_outputs.shape)
print (transformer_test_inputs.shape, subword_test_inputs.shape, word_test_inputs.shape, char_test_inputs.shape, \
       test_outputs.shape)
print (full_transformer_test_inputs.shape, full_subword_test_inputs.shape, full_word_test_inputs.shape, \
       full_char_test_inputs.shape)



100%|██████████| 457/457 [00:00<00:00, 2023.65it/s]

100%|██████████| 457/457 [00:00<00:00, 1942.12it/s]

100%|██████████| 458/458 [00:00<00:00, 1963.43it/s]

100%|██████████| 458/458 [00:00<00:00, 1788.73it/s]

100%|██████████| 4737/4737 [00:02<00:00, 1793.03it/s]

100%|██████████| 4737/4737 [00:02<00:00, 1685.60it/s]


(37867, 200) (37867, 50, 20) (37867, 50) (37867, 200) (37867, 2)
(457, 200) (457, 50, 20) (457, 50) (457, 200) (457, 2)
(458, 200) (458, 50, 20) (458, 50) (458, 200) (458, 2)
(4737, 200) (4737, 50, 20) (4737, 50) (4737, 200)


In [62]:
model = Transformer(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords, n_out=n_out,max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', models.utils.f1_keras]) 

lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, \
                                                  patience=args.lr_schedule_round, verbose=1, mode='auto', min_lr=0.000001)

model_save_path = os.path.join(args.model_save_path,'pseudo_labelling_transformer.h5')

f1callback = models.utils.F1Callback(model, [word_val_inputs, char_val_inputs, subword_val_inputs, transformer_val_inputs],\
                                      val_outputs, \
                                      filename=model_save_path, \
                                      patience=args.early_stopping_rounds)

K.clear_session()
    
model.fit([word_train_inputs, char_train_inputs, subword_train_inputs, transformer_train_inputs], train_outputs, \
                      validation_data=([word_val_inputs, char_val_inputs, subword_val_inputs, transformer_val_inputs], val_outputs), \
                          epochs=args.epochs,batch_size=args.train_batch_size, callbacks=[lr, f1callback], verbose=1)

model.load_weights(model_save_path)

test_pred = model.predict([word_test_inputs, char_test_inputs, subword_test_inputs, transformer_test_inputs])

report = classification_report([idx2label[i] for i in test_outputs.argmax(-1)], \
                               [idx2label[i] for i in test_pred.argmax(-1)])

f1 = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                               [idx2label[i] for i in test_pred.argmax(-1)], average='weighted')

f1_macro = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                               [idx2label[i] for i in test_pred.argmax(-1)], average='macro')

print (report, f1, f1_macro)

Epoch 1/500
1184/1184 [==============================] - ETA: 0s - loss: 0.5258 - accuracy: 0.7473 - f1_keras: 0.6114
Score 0.1814272030651341. Model saved in ../models/hate_detection/pseudo_labelling_transformer.h5.
1184/1184 [==============================] - 450s 380ms/step - loss: 0.5258 - accuracy: 0.7473 - f1_keras: 0.6114 - val_loss: 1.7445 - val_accuracy: 0.1816 - val_f1_keras: 0.1678 - lr: 0.0010
Epoch 2/500
  38/1184 [..............................] - ETA: 7:02 - loss: 0.3654 - accuracy: 0.8388 - f1_keras: 0.7694

KeyboardInterrupt: 